# 問題文

## 問題

RNA 文字列に含まれるエキソン（exon）とイントロン（intron）が識別されていれば、  
**イントロン部分を削除し、エキソンを連結**することで、翻訳（タンパク質への変換）に備えた新しい RNA 配列を得ることができます。

---

## 与えられるもの

- DNA 文字列 `s`（長さ最大 1000 塩基対）
- `s` の部分文字列としての複数のイントロン  
- すべての文字列は FASTA 形式で与えられます。

---

## 求めるもの

- `s` からイントロンをすべて削除し、残りの部分（エキソン）を結合
- それを RNA に転写（T → U）
- さらに翻訳して得られる **タンパク質文字列**

※ 与えられるデータに対しては、常に **一意の翻訳結果**が存在します。

In [6]:
def read_fasta(fasta):
    sequences = {}
    header = None
    seq = []
    fasta = fasta.splitlines()
    for line in fasta:
        if line.startswith('>'):
            if header is not None:
                sequences[header] = ''.join(seq)
            header = line[1:]  # Remove '>'
            seq = []
        else:
            seq.append(line)
    if header is not None:
        sequences[header] = ''.join(seq)  # Add the last sequence
    return sequences

In [7]:
codon_dict = {
    "UUU": "F", "UUC": "F", "UUA": "L", "UUG": "L",
    "CUU": "L", "CUC": "L", "CUA": "L", "CUG": "L",
    "AUU": "I", "AUC": "I", "AUA": "I", "AUG": "M",
    "GUU": "V", "GUC": "V", "GUA": "V", "GUG": "V",
    "UCU": "S", "UCC": "S", "UCA": "S", "UCG": "S",
    "CCU": "P", "CCC": "P", "CCA": "P", "CCG": "P",
    "ACU": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "GCU": "A", "GCC": "A", "GCA": "A", "GCG": "A",
    "UAU": "Y", "UAC": "Y", "UAA": "Stop", "UAG": "Stop",
    "CAU": "H", "CAC": "H", "CAA": "Q", "CAG": "Q",
    "AAU": "N", "AAC": "N", "AAA": "K", "AAG": "K",
    "GAU": "D", "GAC": "D", "GAA": "E", "GAG": "E",
    "UGU": "C", "UGC": "C", "UGA": "Stop", "UGG": "W",
    "CGU": "R", "CGC": "R", "CGA": "R", "CGG": "R",
    "AGU": "S", "AGC": "S", "AGA": "R", "AGG": "R",
    "GGU": "G", "GGC": "G", "GGA": "G", "GGG": "G"
}

In [8]:
def remove_introns(dna, introns):
    for intron in introns:
        dna = dna.replace(intron, '')
    return dna

In [9]:
def translate_rna_to_protein(rna):
    protein = []
    for codon in range(0, len(rna), 3):
        amino_acid = codon_dict.get(rna[codon:codon+3], '')
        if amino_acid == "Stop":
            break
        if amino_acid:
            protein.append(amino_acid)
    return ''.join(protein)

In [10]:
def main(fasta):
    sequences = read_fasta(fasta)
    dna = list(sequences.values())[0]
    introns = list(sequences.values())[1:]
    spliced_dna = remove_introns(dna, introns)
    rna = spliced_dna.replace('T', 'U')
    protein = translate_rna_to_protein(rna)
    return protein

In [11]:
sample = """>Rosalind_10
ATGGTCTACATAGCTGACAAACAGCACGTAGCAATCGGTCGAATCTCGAGAGGCATATGGTCACATGATCGGTCGAGCGTGTTTCAAAGTTTGCGCCTAG
>Rosalind_12
ATCGGTCGAA
>Rosalind_15
ATCGGTCGAGCGTGT"""

In [12]:
main(sample)

'MVYIADKQHVASREAYGHMFKVCA'